In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [7]:
len(df_train), len(df_val)

(2421440, 2918187)

In [8]:
# Feature Engineering
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] 
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
# LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.530253530719832

In [13]:
# lasso
la = Lasso()
la.fit(X_train, y_train)

y_pred = la.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

9.708000985372925

In [14]:
# Ridge
rd = Ridge()
rd.fit(X_train, y_train)

y_pred = rd.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.008353826751957

In [15]:
# GradiantBoosting
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)

y_pred = gb.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.290134916555241

Since `GradiantBoosting` gave the least mean squared error, we will use it for production but we will save all the models used.

In [16]:
from pickle import dump

dump(dv, open('models/DictVectorizer.pkl', 'wb'))
dump(dv, open('models/DictVectorizer.bin', 'wb'))
dump(lr, open('models/lr_model.pkl', 'wb'))
dump(la, open('models/la_model.pkl', 'wb'))
dump(rd, open('models/rd_model.pkl', 'wb'))
dump(gb, open('models/gbr_model.pkl', 'wb'))
dump(gb, open('models/gb_reg.bin', 'wb'))